1. add ipy_checkpoints and pycache to git ignroe and push
2. generalize all functions ie the models, and data writers to handle all batch transformations
3. after rbf, test all setups on collab
3. add images
4. mnist
   add image func for what you get wrong
5. for fashion mnist, use optimal settings, and different hyperparam sets

In [10]:
import os
import itertools
import json
import pickle

import torch
import torch.utils.tensorboard as tb
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.profiler import profile, ProfilerActivity, record_function

import display
import models
import general_torch as gt

In [11]:
# init invariants

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float
cwd = os.getcwd()


# THESE ARE THE THINGS YOU CHANGE
# UPDATE MODELS
# INIT MODEL
# INIT CONTROL AND INVARIANTS
write = True
preprocess = False
experiment_directory = f'experiments/cluster/mlp'
data_path = 'cluster_dict.pt'
# model is module chain, id, output
model = 'mlp1_4'

invariable_dict = {
  'data_path': data_path,
  'exp_dir': experiment_directory,
  'model_type': 'classification',
  'device': device,
  'data_type': dtype,
  'run': None,
  'model': model,
  'init_num': '0',
  'optim': 'sgd',
  'optim_dict': {'lr': 0.1, 'momentum': 0.4},
  'error_f': 'bce',
  'stop_dict': {'criteria': 'epoch', 'stop_val': 0.001, 'max_epochs': 100}, # early stop
  'dl_dict': {'batch_size': 1, 'batch_type':'fixed', 'shuffle': True},  # batch should be a fraction
  'pre_dict': {
      'centers': 5,
      'sigma': 1,
      'cluster_algor': 'kmean',
      'rbf_init': None,
  },
  'seed': 42,
  'hparams': None,
}


In [12]:
# load in data

file_tensor = os.path.join(cwd, f'data/{invariable_dict["data_path"]}')

dict_tensor = torch.load(file_tensor)

train_data = dict_tensor['train_data'].to(device).to(dtype)
train_label = dict_tensor['train_label'].to(device).to(dtype)

valid_data = dict_tensor['valid_data'].to(device).to(dtype)
valid_label = dict_tensor['valid_label'].to(device).to(dtype)

test_data = dict_tensor['test_data'].to(device).to(dtype)
test_label = dict_tensor['test_label'].to(device).to(dtype)

# todo in future init this in data init
class_labels = [i for i in range(int(dict_tensor['num_classes']))]

In [13]:
# init data class and data sets

class Data(Dataset):
  def __init__(self, train, label):
    self.data = train
    self.label = label

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx], self.label[idx]


train_set = Data(train_data, train_label)
valid_set = Data(valid_data, valid_label)
test_set = Data(test_data, test_label)

In [14]:
# use itertools to generate hparam dictionaries

hparam1 = {'model':invariable_dict['model']}
combined_hparams = [hparam1]

'''combined_hparams = [
  {'optim_dict': opt, 'dl_dict': dl, 'error_f': err}
  for opt, dl, err in itertools.product(hparam1['optim_dict'])
]'''

"combined_hparams = [\n  {'optim_dict': opt, 'dl_dict': dl, 'error_f': err}\n  for opt, dl, err in itertools.product(hparam1['optim_dict'])\n]"

In [15]:
# using combined hparams and the invariable base dict, create
# run dict

run_dict = {}
for run, hparam_dict in enumerate(combined_hparams):
  new_dict = {**invariable_dict, **hparam_dict}
  new_dict['hparams'] = hparam_dict
  new_dict['run'] = 0
  dict_pair = {run: new_dict}
  run_dict.update(dict_pair)


In [16]:
# todo in main script, replace this with an encapsulating loop, right now keep it simple
# init loop variables

exp = run_dict[0]

# always start by resetting seeds before initializing anythin  g
gt.init_seeds(exp['seed'], exp['device'])

# prep dataloader, use kwargs to init

if exp['dl_dict']['batch_type'] == 'fixed':
  batch_size = exp['dl_dict']['batch_size']
else:
  batch_fraction = exp['dl_dict']['batch_size']
  batch_size = int(len(train_label) * batch_fraction)
  if batch_size == 0:
    batch_size = 1
exp['dl_dict'] = {'batch_size': batch_size, 'shuffle': exp['dl_dict']['shuffle']}

train_loader = DataLoader(train_set, **exp['dl_dict'])
valid_loader = DataLoader(valid_set, batch_size=len(valid_label), shuffle=False)
test_loader = DataLoader(test_set, batch_size=len(test_label,), shuffle=False)

# build model / load state dict / load whole model / explicitly init param weights

model_path = f"{exp['model']}_{exp['init_num']}"

model = models.get_model(exp['model'])()
model.load_state_dict(models.load_init_model(model_path))
model.to(exp['data_type'])
model.to(exp['device'])

optim = gt.get_optim(exp['optim'])(model.parameters(), **exp['optim_dict'])
error = gt.get_error(exp['error_f'])()

profile_model = models.get_model(exp['model'])()
profile_model.load_state_dict(models.load_init_model(model_path))
profile_model.to(exp['data_type'])
profile_model.to(exp['device'])
profile_train_loader = DataLoader(train_set, **exp['dl_dict'])

# writer doesnt like interior dicts, get rid of them in hparams

format_hparams = {**exp['hparams']}

for key, value in format_hparams.items():
  if isinstance(value, dict):
    format_hparams[key] = json.dumps(value)

# init tensorboard writer
if write:
  exp_dir = os.path.join(cwd, exp['exp_dir'])
  log = os.path.join(exp_dir, f"run{exp['run']}")
  train_writer = tb.writer.SummaryWriter(log_dir=log)
  test_writer = tb.writer.SummaryWriter(log_dir=os.path.join(log, 'test'))

In [17]:
# init if rbf

from sklearn.cluster import KMeans

if preprocess:
    kmeans = KMeans(n_clusters=exp['pre_dict']['centers'],
                    random_state=exp['seed'])
    kmeans.fit(train_data.numpy())
    sigmas = [exp['pre_dict']['sigma']] * exp['pre_dict']['centers']
    model.post_init(kmeans.cluster_centers_, sigmas)
    

In [18]:
# check for runtime errors / init predictions should be part of data collection loop
'''for i in model.parameters():
    print(i.dtype)
    break
print(test_data.dtype)'''

with torch.inference_mode():
  for batch_idx, (data, target) in enumerate(train_loader):
    model(data)

In [19]:
# profile model

'''with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
             schedule=torch.profiler.schedule(wait=5, warmup=4, active=1, repeat=1),
             on_trace_ready=torch.profiler.tensorboard_trace_handler(os.path.join(log, 'model_profiles')),
             record_shapes=True,
             with_stack=True,
             profile_memory=True) as prof:
  with record_function('training'):
    for batch_idx, (data, target) in enumerate(train_loader):
      pred = model(data)
      loss = error(pred, target)
      optim.zero_grad()
      loss.backward()
      optim.step()
      prof.step()
      alpha = models.newclass.loop_alpha(batch_idx)
      if batch_idx == 9:
        break

  with record_function('inference'):
    for batch_idx, (data, target) in enumerate(test_loader):
      model(data)
      prof.step()
      if batch_idx == 9:
        break'''


"with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],\n             schedule=torch.profiler.schedule(wait=5, warmup=4, active=1, repeat=1),\n             on_trace_ready=torch.profiler.tensorboard_trace_handler(os.path.join(log, 'model_profiles')),\n             record_shapes=True,\n             with_stack=True,\n             profile_memory=True) as prof:\n  with record_function('training'):\n    for batch_idx, (data, target) in enumerate(train_loader):\n      pred = model(data)\n      loss = error(pred, target)\n      optim.zero_grad()\n      loss.backward()\n      optim.step()\n      prof.step()\n      alpha = models.newclass.loop_alpha(batch_idx)\n      if batch_idx == 9:\n        break\n\n  with record_function('inference'):\n    for batch_idx, (data, target) in enumerate(test_loader):\n      model(data)\n      prof.step()\n      if batch_idx == 9:\n        break"

In [20]:
# init test predictions, to test overall improvement, only needed if model reuse

'''with torch.inference_mode():
    pred_list = []
    target_list = []
    total = 0
    for batch_idx, (data, target) in enumerate(train_loader):

        preds = model(data)
        loss = error(preds, target)
        pred_list.append(preds)
        target_list.append(target)
        total += loss
    avg_error = total / len(test_data)
    gt.write_test_info(exp, train_writer, target_list, pred_l
ist, avg_error)'''

'with torch.inference_mode():\n    pred_list = []\n    target_list = []\n    total = 0\n    for batch_idx, (data, target) in enumerate(train_loader):\n\n        preds = model(data)\n        loss = error(preds, target)\n        pred_list.append(preds)\n        target_list.append(target)\n        total += loss\n    avg_error = total / len(test_data)\n    gt.write_test_info(exp, train_writer, target_list, pred_l\nist, avg_error)'

In [21]:
# modify hyperparams, only for model reuse

'''modify_hyper = False
if modify_hyper:
    epochs = 10
    optim = torch.optim.SGD(model.parameters(), lr=0.001, mom
entum=0.4)
    error = torch.nn.MSELoss()'''

'modify_hyper = False\nif modify_hyper:\n    epochs = 10\n    optim = torch.optim.SGD(model.parameters(), lr=0.001, mom\nentum=0.4)\n    error = torch.nn.MSELoss()'

In [22]:
# for train and valid, log every epoch

error_last = None
for epoch in range(exp['stop_dict']['max_epochs']):
  model.train()
  total = 0
  target_list_t = []
  pred_list_t = []
  for batch_idx, (data, target) in enumerate(train_loader):
    pred = model(data)
    loss = error(pred, target)
    optim.zero_grad()
    loss.backward()
    optim.step()
    if write:
      total += loss
      target_list_t.append(target)
      pred_list_t.append(pred)
  avg_error_t = total / len(train_data)
  # todo push batch unpacking to to the writing as to

  model.eval()
  error_new = None
  total = 0
  target_list = []
  pred_list = []
  for batch_idx, (data, target) in enumerate(valid_loader):
    pred = model(data)
    loss = error(pred, target)
    total += loss
    if write:
      target_list.append(target)
      pred_list.append(pred)
  avg_error = total / len(valid_data)
  
  if write:
    display.write_classification_train_info(train_writer, model, target_list_t, pred_list_t, avg_error_t, 'train', epoch)
    display.write_classification_train_info(train_writer, model, target_list, pred_list, avg_error, 'valid', epoch)

  error_new = avg_error
  if exp['stop_dict']['criteria'] == 'early_stop':
    if error_last is not None and error_new / error_last < exp['stop_dict']['stop_val']:
      break
    else:
      error_last = error_new


In [23]:
# run final inference

with torch.inference_mode():
  pred_tensor_list = []
  target_tensor_list = []
  total = 0
  for batch_idx, (data, target) in enumerate(test_loader):
    preds = model(data)
    loss = error(preds, target)
    pred_tensor_list.append(preds)
    target_tensor_list.append(target)
    total += loss
  avg_error = total / len(test_data)

In [24]:
# do final tensorboard run write
# save preds, labels and state dict
# write test info into db
cm = display.plot_multiclass_confusion_matrix(target_tensor_list, pred_tensor_list, class_labels)
decision_boundary = display.plot_decision_boundary(exp, model, test_data, test_label)
print(exp)

if write:
  train_writer.add_graph(model, train_data)
  file_dir = {'preds': pred_list, 'lables': target_list,
              'state_dict': model.state_dict(), }
  filename = os.path.join(log, 'model_info.pkl')

  with open(filename, "wb") as file:
    pickle.dump(file_dir, file)

  train_writer.add_figure('test/confusion_matrix', cm)
  train_writer.add_figure('test/decision_boundary', decision_boundary)
  display.write_classification_test_info(test_writer, format_hparams, target_tensor_list, pred_tensor_list, avg_error)
  # train_writer.add_pr_curve('test/precision_recall_curve', target_tensor, pred_tensor, 0)
  train_writer.close()
  test_writer.close()

{'data_path': 'cluster_dict.pt', 'exp_dir': 'experiments/cluster/mlp', 'model_type': 'classification', 'device': 'cpu', 'data_type': torch.float32, 'run': 0, 'model': 'mlp1_4', 'init_num': '0', 'optim': 'sgd', 'optim_dict': {'lr': 0.1, 'momentum': 0.4}, 'error_f': 'bce', 'stop_dict': {'criteria': 'epoch', 'stop_val': 0.001, 'max_epochs': 100}, 'dl_dict': {'batch_size': 1, 'shuffle': True}, 'pre_dict': {'centers': 5, 'sigma': 1, 'cluster_algor': 'kmean', 'rbf_init': None}, 'seed': 42, 'hparams': {'model': 'mlp1_4'}}
